In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")  # Go up one level to the UROP directory

In [2]:
import yaml

with open("config.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

batch_size = config["batch_size"]
lr = config["lr"]
num_epochs = config["num_epochs"]
save_every = config["save_every"]
save_dir = config["save_dir"]
num_workers = config["num_workers"]

In [3]:
import torch
from torch.utils.data import DataLoader
from src import LazyWindowedDataset

test_source_dataset = LazyWindowedDataset(
    root_dir="datasets/RoboticArm",
    split="test",
    anomaly_type=['normal', 'anomaly'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)
test_loader = DataLoader(test_source_dataset, batch_size=batch_size, shuffle=True, num_workers=1, drop_last=True)

In [4]:
from models import BaselineAutoencoder
from src import evaluate, load_model_checkpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, _, _, _, _, _ = load_model_checkpoint("checkpoints/RoboticArmBaseline.pt", BaselineAutoencoder, optimizer_class=None)
criterion = torch.nn.MSELoss()

In [5]:
from torchinfo import summary
summary(model, (batch_size, 5620))

Layer (type:depth-idx)                   Output Shape              Param #
BaselineAutoencoder                      [128, 5620]               --
├─Sequential: 1-1                        [128, 16]                 --
│    └─Linear: 2-1                       [128, 2048]               11,511,808
│    └─ReLU: 2-2                         [128, 2048]               --
│    └─Linear: 2-3                       [128, 2048]               4,196,352
│    └─ReLU: 2-4                         [128, 2048]               --
│    └─Linear: 2-5                       [128, 2048]               4,196,352
│    └─ReLU: 2-6                         [128, 2048]               --
│    └─Linear: 2-7                       [128, 16]                 32,784
├─Sequential: 1-2                        [128, 5620]               --
│    └─Linear: 2-8                       [128, 2048]               34,816
│    └─ReLU: 2-9                         [128, 2048]               --
│    └─Linear: 2-10                      [128, 2048]   

![alt text](figures/roboticArmHighlightedTable.png)

In [ ]:
loss, auc = evaluate(model, test_loader, criterion, verbose=1)
print(f"Overall S+T | Loss: {loss:.4f}, AUC: {auc:.4f}")

Overall S+T | Loss: 0.6323, AUC: 0.8988


In [ ]:
acc_loss, acc_auc = evaluate(model, test_loader, criterion, sensors_to_test=['acc'], verbose=1)
print(f"Acc S+T | Loss: {acc_loss:.4f}, AUC: {acc_auc:.4f}")

Acc s+T | Loss: 0.1346, AUC: 0.8576


In [ ]:
gyro_loss, gyro_auc = evaluate(model, test_loader, criterion, sensors_to_test=['gyro'], verbose=1)
print(f"Gyro S+T | Loss: {gyro_loss:.4f}, AUC: {gyro_auc:.4f}")

Gyro S+T | Loss: 1.4949, AUC: 0.6337


In [ ]:
mic_loss, mic_auc = evaluate(model, test_loader, criterion, sensors_to_test=['mic'], verbose=1)
print(f"Mic S+T | Loss: {mic_loss:.4f}, AUC: {mic_auc:.4f}")

Mic S+T | Loss: 1.0299, AUC: 0.5433
